In [1]:
import os
%pwd

'/Users/hh/MLops/nlp_project/research'

In [2]:
os.chdir('../')
%pwd

'/Users/hh/MLops/nlp_project'

In [3]:

!pip uninstall -y transformers accelerate

!pip install -U torch

!pip uninstall -y transformers
!pip cache purge -y || true
!pip install -U "transformers[torch]" "accelerate>=0.26.0"



Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0

Usage:   
  pip cache dir
  pip cache info
  pip cache list [<pattern>] [--format=[human, abspath]]
  pip cache remove <pattern>
  pip cache purge
  

no such option: -y
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradiant_accumulation_steps: int 


In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
class ConfigurationManager():
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params=self.params.TrainingArguments
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradiant_accumulation_steps= params.gradiant_accumulation_steps
        )
        return model_trainer_config

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch, sys
from datasets import load_from_disk


/Users/hh/MLops/nlp_project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class ModelTrainer:
    def __init__(self, model_trainer_config: ModelTrainerConfig):
        self.config = model_trainer_config

    def train(self):
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        data_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            eval_steps=self.config.eval_steps, save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradiant_accumulation_steps,

            no_cuda=True,
            use_mps_device=False,

        )

        trainer = Trainer(model=model_pegasus, args=trainer_args, tokenizer=tokenizer,
                          data_collator=seq2seq_data_collator, train_dataset=data_samsum_pt['test'],
                          eval_dataset=data_samsum_pt['validation'])
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

        


In [8]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer_config = ModelTrainer(model_trainer_config)
model_trainer_config.train()

[2025-12-08 03:55:49,354] 13 src.textSummarizer.logging - INFO - entered read_yaml function at /Users/hh/MLops/nlp_project
[2025-12-08 03:55:49,357] 16 src.textSummarizer.logging - INFO - yaml file: config/config.yaml loaded
[2025-12-08 03:55:49,358] 13 src.textSummarizer.logging - INFO - entered read_yaml function at /Users/hh/MLops/nlp_project
[2025-12-08 03:55:49,359] 16 src.textSummarizer.logging - INFO - yaml file: params.yaml loaded
[2025-12-08 03:55:49,360] 29 src.textSummarizer.logging - INFO - created directory at artifacts
[2025-12-08 03:55:49,360] 29 src.textSummarizer.logging - INFO - created directory at artifacts/model_trainer


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/hh/MLops/nlp_project/venv/lib/python3.10/site-packages/transformers/training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/var/folders/1z/knshtkp96gv94c7gjtb06jkh0000gn/T/ipykernel_94142/4200769611.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args, tokenizer=tokenizer,
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and g

Step,Training Loss
10,3.167300
20,3.169000
30,3.107700
40,2.961800
50,2.954300


/Users/hh/MLops/nlp_project/venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
